# 井点属性自动提取 + 伪样本生成


In [ ]:
# 确保src目录在Python路径中
import os
import sys

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scipy.stats import pearsonr, spearmanr
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LassoCV, LinearRegression, RidgeCV
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.model_selection import GridSearchCV, LeaveOneOut
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVR
from xgboost import XGBRegressor

sys.path.append(os.path.abspath("../"))

# 导入模块
from src.data_utils import (
    extract_seismic_attributes_for_wells,
    extract_uniform_seismic_samples,
    filter_anomalous_attributes,
    filter_outlier_wells,
    filter_seismic_by_wells,
    identify_attributes,
    parse_petrel_file,
    preprocess_features,
)
from src.feature_selection import select_best_features
from src.visualization import visualize_attribute_map

data_dir = "..\\data"
output_dir = "output"
if not os.path.exists(output_dir):
    os.makedirs(output_dir)


# 设置中文字体
plt.rcParams["font.family"] = "SimHei"  # 黑体 SimHei 支持中文
plt.rcParams["axes.unicode_minus"] = False  # 正常显示负号

## 导入地震数据


In [ ]:
data_seismic_attr = parse_petrel_file(os.path.join(data_dir, "H6-2_attr"))

## 导入井点位置


In [ ]:
data_well_position = pd.read_excel(os.path.join(data_dir, "well_without_attr.xlsx"))

# 选择对应层位的行，丢弃砂厚为 NaN 的行
data_well_purpose_surface_position = (
    data_well_position[data_well_position["Surface"] == "H6-2"]
    .replace(-999, np.nan)  # 将-999替换为NaN
    .dropna(subset=["Sand Thickness"])  # 丢弃砂厚为NaN的行
    .reset_index(drop=True)  # 重置索引
)
data_well_purpose_surface_position.head()

## 筛除离群井


In [ ]:
# 筛选离群井
data_well_purpose_surface_filtered = filter_outlier_wells(data_well_purpose_surface_position, method="iqr")

# 显示筛选前后的井点数量
print(f"筛选前井点数量: {len(data_well_purpose_surface_position)}")
print(f"筛选后井点数量: {len(data_well_purpose_surface_filtered)}")

# 可视化筛选前后的井点分布
plt.figure(figsize=(12, 6))

# 计算坐标范围（使用所有井点的数据来确定范围）
x_min = data_well_purpose_surface_position["X"].min()
x_max = data_well_purpose_surface_position["X"].max()
y_min = data_well_purpose_surface_position["Y"].min()
y_max = data_well_purpose_surface_position["Y"].max()

# 可选：添加一些边距使图更美观
margin = 0.05  # 5%的边距
x_range = x_max - x_min
y_range = y_max - y_min
x_min -= x_range * margin
x_max += x_range * margin
y_min -= y_range * margin
y_max += y_range * margin

# 绘制筛选前的井点分布
plt.subplot(1, 2, 1)
plt.scatter(data_well_purpose_surface_position["X"], data_well_purpose_surface_position["Y"], c="blue")
plt.title("筛选前井点分布")
plt.xlabel("X坐标")
plt.ylabel("Y坐标")
plt.xlim(x_min, x_max)
plt.ylim(y_min, y_max)

# 绘制筛选后的井点分布
plt.subplot(1, 2, 2)
plt.scatter(data_well_purpose_surface_filtered["X"], data_well_purpose_surface_filtered["Y"], c="red")
plt.title("筛选后井点分布")
plt.xlabel("X坐标")
plt.ylabel("Y坐标")
plt.xlim(x_min, x_max)
plt.ylim(y_min, y_max)

plt.tight_layout()
plt.savefig(os.path.join(output_dir, "well_filtering_comparison.png"))
plt.show()

## 处理属性缺失值


In [ ]:
# 首先获取地震属性列表
attribute_names, _ = identify_attributes(os.path.join(data_dir, "H6-2_attr"))

# 使用preprocess_features处理地震数据
processed_seismic, attr_stats = preprocess_features(
    data=data_seismic_attr,
    attribute_columns=attribute_names,
    missing_values=[-999],
    missing_threshold=0.6,  # 缺失值超过60%的列将被删除
    outlier_method="iqr",
    outlier_threshold=1.5,
    verbose=True,
)

# 提取筛选后的属性
attribute_names_filtered = [col for col in processed_seismic.columns]

# 将处理后的属性数据与原始坐标数据合并
processed_seismic_full = data_seismic_attr[["X", "Y"]].copy()
for col in processed_seismic.columns:
    processed_seismic_full[col] = processed_seismic[col]

## 根据井点分布，缩小工区范围


In [ ]:
# 限制工区范围
seismic_attr_filtered, area_bounds = filter_seismic_by_wells(
    seismic_data=processed_seismic_full,
    well_data=data_well_purpose_surface_filtered,
    expansion_factor=1.5,  # 扩展50%
    plot=True,
    output_dir=output_dir,
)

# 后续可以直接使用area_bounds中的边界信息
print("区域边界信息:")
for key, value in area_bounds.items():
    print(f"  {key}: {value}")

## 提取井点处地震属性


In [ ]:
# 为筛选前的井点提取地震属性
well_attr = extract_seismic_attributes_for_wells(
    well_data=data_well_purpose_surface_position,
    seismic_data=processed_seismic_full,
    max_distance=50,
    num_points=5,
)

# 为筛选后的井点提取地震属性
well_attr_filtered = extract_seismic_attributes_for_wells(
    well_data=data_well_purpose_surface_filtered, seismic_data=processed_seismic_full, max_distance=50, num_points=5
)

# 保存处理结果
well_attr.to_excel(os.path.join(data_dir, "wells_attr.xlsx"), index=False)
print("筛选前井点的地震属性已保存到 wells_attr.xlsx")
well_attr_filtered.to_excel(os.path.join(data_dir, "wells_attr_filtered.xlsx"), index=False)
print("筛选后井点的地震属性已保存到 wells_attr_filtered.xlsx")

## 生成统计摘要


In [ ]:
# 筛选出质量良好的属性
good_attributes, anomalous_attributes, attribute_stats = filter_anomalous_attributes(
    seismic_data=seismic_attr_filtered,
    well_data=well_attr_filtered,
    common_attributes=attribute_names_filtered,
    ratio_threshold=5.0,  # 均值比值阈值
    range_ratio_threshold=10.0,  # 数值范围比值阈值
    std_ratio_threshold=10.0,  # 标准差比值阈值
    output_dir=None,  # 输出图表目录
    verbose=True,  # 打印详细信息
)

print("\n筛选后保留的质量良好属性:")
for attr in good_attributes:
    print(f"- {attr}")

## 随机森林重要性和相关性分析


In [ ]:
# 使用随机森林评估特征重要性并移除冗余特征
selected_features = select_best_features(
    well_data=well_attr_filtered,
    attribute_columns=good_attributes,
    target_column="Sand Thickness",
    n_features=3,
    corr_threshold=0.85,
    output_dir=output_dir,
    verbose=True,
)

# 输出特征选择结果
print("\n基于随机森林重要性和相关性分析的最佳特征:")
for i, feature in enumerate(selected_features):
    print(f"{i + 1}. {feature}")

## 提取样本，准备设置虚拟井


In [ ]:
# 使用筛选后的地震数据区域提取等间距样本
seismic_samples = extract_uniform_seismic_samples(
    seismic_data=seismic_attr_filtered,
    n_rows=40,
    n_cols=40,
    area_bounds=area_bounds,  # 直接传入边界字典
)

# 可视化真实井点和采样点
plt.figure(figsize=(15, 10))

# 绘制地震数据点（使用抽样）
sample_ratio = min(1.0, 5000 / len(seismic_attr_filtered))
seismic_sample = seismic_attr_filtered.sample(frac=sample_ratio)
plt.scatter(seismic_sample["X"], seismic_sample["Y"], color="lightgray", alpha=0.3, s=10, label="地震数据(抽样)")

# 绘制真实井点位置
plt.scatter(well_attr_filtered["X"], well_attr_filtered["Y"], color="red", s=100, marker="^", label="真实井点")

# 绘制等间距采样点位置
plt.scatter(seismic_samples["X"], seismic_samples["Y"], color="blue", s=50, marker="o", label="等间距采样点")

# 添加标题和图例
plt.title("真实井点与等间距采样点分布", fontsize=16)
plt.xlabel("X坐标", fontsize=14)
plt.ylabel("Y坐标", fontsize=14)
plt.legend(loc="upper right")
plt.grid(True, linestyle="--", alpha=0.7)

# 保存图片
plt.savefig(os.path.join(output_dir, "real_wells_and_seismic_samples.png"), dpi=300, bbox_inches="tight")
plt.show()


# 保存提取的样本数据
seismic_samples.to_csv(os.path.join(output_dir, "seismic_samples.csv"), index=False)
print(f"等间距地震样本数据已保存至 {os.path.join(output_dir, 'seismic_samples.csv')}")

## 多线性模型一致性预测设置虚拟井


In [ ]:
# 创建融合属性和多模型预测
print("======== 创建融合属性和多模型预测 ========")
target_column = "Sand Thickness"
min_corr_threshold = 0.3  # 最小相关性阈值，低于此值的属性将被排除

# 准备训练数据
X_labeled = well_attr_filtered[selected_features].values
y_labeled = well_attr_filtered[target_column].values

# 准备未标记数据（地震样本点）
X_unlabeled = seismic_samples[selected_features].dropna().values
unlabeled_indices = seismic_samples[selected_features].dropna().index

# 1. 融合属性线性加权预测
print("\n=== 模型1: 融合属性线性加权 ===")

# 计算相关性权重
correlation_weights = {}
for i, feature in enumerate(selected_features):
    corr, _ = pearsonr(well_attr_filtered[feature], well_attr_filtered[target_column])
    if abs(corr) >= min_corr_threshold:
        correlation_weights[feature] = corr
        print(f"属性 '{feature}' 与砂厚的 Pearson 相关性: {corr:.4f}")
    else:
        print(f"属性 '{feature}' 与砂厚的相关性过低 ({corr:.4f})，不纳入融合")

# 如果没有有效属性，使用所有属性且权重相等
if len(correlation_weights) == 0:
    print("警告: 没有属性满足相关性阈值，将使用所有属性且权重相等")
    for feature in selected_features:
        correlation_weights[feature] = 1.0
        print(f"属性 '{feature}' 使用默认权重: 1.0")

# 标准化数据
scaler = StandardScaler()
X_labeled_scaled = scaler.fit_transform(X_labeled)
X_unlabeled_scaled = scaler.transform(X_unlabeled)


# 创建融合属性函数
def create_fused_attribute(X_scaled, features, weights):
    """
    基于选定特征和权重创建融合属性

    参数:
        X_scaled (ndarray): 标准化后的特征矩阵
        features (list): 特征列表
        weights (dict): 每个特征的权重

    返回:
        ndarray: 融合属性
    """
    # 初始化融合属性
    fused_attr = np.zeros(X_scaled.shape[0])
    weight_sum = 0

    # 对每个特征进行加权融合
    for i, feature in enumerate(features):
        if feature in weights:
            weight = weights[feature]
            fused_attr += X_scaled[:, i] * weight
            weight_sum += abs(weight)

    # 归一化融合结果
    if weight_sum > 0:
        fused_attr /= weight_sum

    return fused_attr


# 在训练数据上创建融合属性
fused_attr_labeled = create_fused_attribute(X_labeled_scaled, selected_features, correlation_weights)

# 在未标记数据上创建融合属性
fused_attr_unlabeled = create_fused_attribute(X_unlabeled_scaled, selected_features, correlation_weights)

# 使用融合属性拟合线性回归
linear_model = LinearRegression()
linear_model.fit(fused_attr_labeled.reshape(-1, 1), y_labeled)

# 预测
fused_pred_labeled = linear_model.predict(fused_attr_labeled.reshape(-1, 1))
fused_pred_unlabeled = linear_model.predict(fused_attr_unlabeled.reshape(-1, 1))

# 评估融合属性预测效果
fused_corr = np.corrcoef(fused_pred_labeled, y_labeled)[0, 1]
print(f"融合属性预测结果与真实砂厚的相关性: {fused_corr:.4f}")

# 2. Lasso回归 + Bootstrap
print("\n=== 模型2: LassoCV + Bootstrap ===")

n_bootstrap = 100  # Bootstrap重采样次数
alpha_values = np.logspace(-4, 1, 30)  # alpha候选值

# 主Lasso模型
lasso_model = LassoCV(alphas=alpha_values, cv=5, max_iter=10000, tol=1e-3)
lasso_model.fit(X_labeled_scaled, y_labeled)
print(f"Lasso最优alpha值: {lasso_model.alpha_:.6f}")

# 在训练数据上的预测
lasso_pred_labeled = lasso_model.predict(X_labeled_scaled)
lasso_corr = np.corrcoef(lasso_pred_labeled, y_labeled)[0, 1]
print(f"Lasso预测结果与真实砂厚的相关性: {lasso_corr:.4f}")

# Bootstrap重采样预测
lasso_bootstrap_preds = np.zeros((n_bootstrap, X_unlabeled_scaled.shape[0]))

for i in range(n_bootstrap):
    # Bootstrap重采样
    indices = np.random.choice(len(X_labeled_scaled), len(X_labeled_scaled), replace=True)
    X_boot, y_boot = X_labeled_scaled[indices], y_labeled[indices]

    # 拟合模型
    lasso_boot = LassoCV(alphas=alpha_values, cv=5, max_iter=10000, tol=1e-3)
    lasso_boot.fit(X_boot, y_boot)

    # 预测
    lasso_bootstrap_preds[i, :] = lasso_boot.predict(X_unlabeled_scaled)

# 计算预测均值和置信区间
lasso_pred_unlabeled = np.mean(lasso_bootstrap_preds, axis=0)
lasso_lower_ci = np.percentile(lasso_bootstrap_preds, 2.5, axis=0)
lasso_upper_ci = np.percentile(lasso_bootstrap_preds, 97.5, axis=0)

# 3. Ridge回归 + Bootstrap
print("\n=== 模型3: RidgeCV + Bootstrap ===")

alpha_values = np.logspace(-3, 3, 30)  # alpha候选值

# 主Ridge模型
ridge_model = RidgeCV(alphas=alpha_values, cv=5)
ridge_model.fit(X_labeled_scaled, y_labeled)
print(f"Ridge最优alpha值: {ridge_model.alpha_:.6f}")

# 在训练数据上的预测
ridge_pred_labeled = ridge_model.predict(X_labeled_scaled)
ridge_corr = np.corrcoef(ridge_pred_labeled, y_labeled)[0, 1]
print(f"Ridge预测结果与真实砂厚的相关性: {ridge_corr:.4f}")

# Bootstrap重采样预测
ridge_bootstrap_preds = np.zeros((n_bootstrap, X_unlabeled_scaled.shape[0]))

for i in range(n_bootstrap):
    # Bootstrap重采样
    indices = np.random.choice(len(X_labeled_scaled), len(X_labeled_scaled), replace=True)
    X_boot, y_boot = X_labeled_scaled[indices], y_labeled[indices]

    # 拟合模型
    ridge_boot = RidgeCV(alphas=alpha_values, cv=5)
    ridge_boot.fit(X_boot, y_boot)

    # 预测
    ridge_bootstrap_preds[i, :] = ridge_boot.predict(X_unlabeled_scaled)

# 计算预测均值和置信区间
ridge_pred_unlabeled = np.mean(ridge_bootstrap_preds, axis=0)
ridge_lower_ci = np.percentile(ridge_bootstrap_preds, 2.5, axis=0)
ridge_upper_ci = np.percentile(ridge_bootstrap_preds, 97.5, axis=0)

## 平衡区间分布与质量控制的伪样本筛选


In [ ]:
def calculate_quality_score(mean_pred, max_diff, rel_diff, lasso_std, ridge_std, thresholds):
    """
    计算样本的质量分数（0-1，越高越好）

    参数:
        mean_pred: 平均预测值
        max_diff: 最大模型差异
        rel_diff: 相对差异
        lasso_std: Lasso模型标准差
        ridge_std: Ridge模型标准差
        thresholds: 包含各阈值的字典
    """
    # 基础权重
    w_consistency = thresholds["w_consistency"]  # 一致性权重
    w_uncertainty = thresholds["w_uncertainty"]  # 不确定性权重

    # 计算一致性得分 (0-1)
    abs_score = max(0, 1 - max_diff / thresholds["abs_threshold"])
    rel_score = max(0, 1 - rel_diff / thresholds["rel_threshold"])
    consistency_score = 0.5 * abs_score + 0.5 * rel_score

    # 计算不确定性得分 (0-1)
    lasso_std_score = max(0, 1 - lasso_std / thresholds["std_threshold"])
    ridge_std_score = max(0, 1 - ridge_std / thresholds["std_threshold"])
    uncertainty_score = 0.5 * lasso_std_score + 0.5 * ridge_std_score

    # 组合得分
    final_score = w_consistency * consistency_score + w_uncertainty * uncertainty_score

    return final_score


def calculate_j_score(selected_indices, quality_scores, mean_preds, target_dist, bins, thresholds):
    """计算当前选择的J评分"""
    if len(selected_indices) == 0:
        return 0

    # 1. 计算质量分数
    selected_quality = quality_scores[selected_indices]
    Q_total = np.mean(selected_quality)

    # 2. 计算分布相似度
    selected_values = mean_preds[selected_indices]
    hist, _ = np.histogram(selected_values, bins=bins)
    current_dist = hist / (np.sum(hist) + 1e-10)

    # Jensen-Shannon散度
    from scipy.spatial.distance import jensenshannon

    jsd = jensenshannon(current_dist, target_dist)
    dist_similarity = 1 - jsd

    # 3. 计算样本数量得分
    n_samples = len(selected_indices)
    min_samples = 200
    max_samples = 1000
    quantity_score = min(1.0, max(0.0, (n_samples - min_samples) / (max_samples - min_samples)))

    # 计算J值 - 使用动态权重
    J = (
        thresholds["j_quality_weight"] * Q_total
        + thresholds["j_dist_weight"] * dist_similarity
        + thresholds["j_quantity_weight"] * quantity_score
    )

    return J

In [ ]:
def get_dynamic_quality_threshold(bin_idx, sample_ratio, base_threshold):
    """
    基于区间特性和充足率的精细调整

    参数:
        bin_idx: 区间索引
        sample_ratio: 区间的当前样本充足率
        base_threshold: 当前基础质量阈值
    """
    # 1. 区间特性调整
    if bin_idx >= 2:  # 高砂厚区间对质量更加宽容
        adjusted = max(0.3, base_threshold - 0.1)
    else:
        adjusted = base_threshold

    # 2. 样本充足率调整
    if sample_ratio < 0.3:  # 严重不足
        return max(0.3, adjusted - 0.1)
    elif sample_ratio < 0.6:  # 较为不足
        return max(0.35, adjusted - 0.05)
    else:  # 接近或超过目标
        return adjusted


def adjust_thresholds(thresholds, iteration, bin_coverage):
    """
    根据迭代情况全局调整下一轮迭代的阈值

    参数:
        thresholds: 当前阈值字典
        iteration: 当前迭代次数
        bin_coverage: 各区间的样本覆盖率
    """
    new_thresholds = thresholds.copy()

    # 1. 根据迭代次数调整模型差异容忍度参数
    iteration_factor = min(1.5, 1.0 + iteration * 0.05)  # 每次迭代增加5%，最多增加50%
    new_thresholds["abs_threshold"] *= iteration_factor
    new_thresholds["rel_threshold"] *= iteration_factor
    new_thresholds["std_threshold"] *= iteration_factor

    # 2. 根据迭代次数调整基础质量阈值
    # 随着迭代进行，逐渐降低基础质量要求
    if iteration > 0:
        new_quality = max(0.3, thresholds["min_quality"] * (1 - iteration * 0.05))
        new_thresholds["min_quality"] = new_quality

    # 3. 根据总体覆盖情况调整J评分权重
    min_coverage = np.min(bin_coverage)
    if min_coverage < 0.3:  # 覆盖率极低
        # 降低分布权重，更关注质量和数量
        new_thresholds["j_dist_weight"] *= 0.85
        new_thresholds["j_quality_weight"] = 1 - new_thresholds["j_dist_weight"] - new_thresholds["j_quantity_weight"]

    # 保证权重和为1
    total_weight = (
        new_thresholds["j_quality_weight"] + new_thresholds["j_dist_weight"] + new_thresholds["j_quantity_weight"]
    )
    if total_weight != 1.0:
        new_thresholds["j_quality_weight"] /= total_weight
        new_thresholds["j_dist_weight"] /= total_weight
        new_thresholds["j_quantity_weight"] /= total_weight

    # 4. 调整J评分容忍度
    new_thresholds["j_tolerance"] = min(0.1, 0.01 + iteration * 0.02)  # 每次迭代增加0.02，最大0.1

    return new_thresholds

In [ ]:
# 4. 平衡分布与质量控制的优化采样
print("\n=== 平衡分布与质量控制的优化采样 ===")

# 将三个模型的预测结果整合
predictions = np.column_stack([fused_pred_unlabeled, lasso_pred_unlabeled, ridge_pred_unlabeled])

# 计算每个点三个预测值的最大差异
max_diffs = np.max(predictions, axis=1) - np.min(predictions, axis=1)

# 计算平均预测值用于相对差异计算
mean_preds = np.mean(predictions, axis=1)

# 预先处理预测值，将负值替换为0
mean_preds = np.maximum(mean_preds, 0)
fused_pred_unlabeled = np.maximum(fused_pred_unlabeled, 0)
lasso_pred_unlabeled = np.maximum(lasso_pred_unlabeled, 0)
ridge_pred_unlabeled = np.maximum(ridge_pred_unlabeled, 0)

# 计算bootstrap结果的标准差
lasso_std = np.std(lasso_bootstrap_preds, axis=0)
ridge_std = np.std(ridge_bootstrap_preds, axis=0)

# 定义砂厚区间
bins = [0, 0.1, 10, np.inf]
bin_labels = ["0-0.1", "0.1-10", ">10"]

# 排除靠近真实井点且砂厚差异大的点
print("\n预处理：排除靠近真实井点且砂厚差异大的点...")
valid_unlabeled_mask = np.ones(len(unlabeled_indices), dtype=bool)
proximity_radius = 100  # 米，设置排除半径
max_thickness_diff = 0.1  # 米，最大允许砂厚差异

# 获取真实井点的坐标和砂厚
real_well_x = well_attr_filtered["X"].values
real_well_y = well_attr_filtered["Y"].values
real_well_thickness = well_attr_filtered["Sand Thickness"].values

# 获取未标记点的坐标
unlabeled_x = seismic_samples.loc[unlabeled_indices, "X"].values
unlabeled_y = seismic_samples.loc[unlabeled_indices, "Y"].values

excluded_count = 0
excluded_reasons = []

# 逐个检查未标记点与真实井点的距离和砂厚差异
for i in range(len(unlabeled_indices)):
    for j in range(len(real_well_x)):
        # 计算距离
        distance = np.sqrt((unlabeled_x[i] - real_well_x[j]) ** 2 + (unlabeled_y[i] - real_well_y[j]) ** 2)

        # 如果距离小于排除半径
        if distance <= proximity_radius:
            # 计算砂厚差异
            thickness_diff = abs(mean_preds[i] - real_well_thickness[j])

            # 如果差异超过阈值，排除该点
            if thickness_diff > max_thickness_diff:
                valid_unlabeled_mask[i] = False
                excluded_count += 1
                excluded_reasons.append(
                    f"点{i}(预测砂厚={mean_preds[i]:.2f})与井点{j}(真实砂厚={real_well_thickness[j]:.2f})距离{distance:.1f}米，差异{thickness_diff:.2f}米"
                )
                break  # 一旦与任何真实井点冲突，就排除该点

if excluded_count > 0:
    print(f"  排除了 {excluded_count} 个靠近真实井点且砂厚差异大的点")
    if excluded_count <= 10:  # 只显示少量详情以避免日志过长
        for reason in excluded_reasons:
            print(f"  - {reason}")
    else:
        print(f"  - 显示前10个排除原因:")
        for reason in excluded_reasons[:10]:
            print(f"    {reason}")

# 更新有效的未标记索引和预测
valid_unlabeled_indices = unlabeled_indices[valid_unlabeled_mask]
valid_mean_preds = mean_preds[valid_unlabeled_mask]
valid_max_diffs = max_diffs[valid_unlabeled_mask]
valid_lasso_std = lasso_std[valid_unlabeled_mask]
valid_ridge_std = ridge_std[valid_unlabeled_mask]
valid_rel_diffs = valid_max_diffs / (valid_mean_preds + 1e-10)  # 避免除零

print(f"  预处理后剩余 {len(valid_unlabeled_indices)} 个有效点 (排除了 {excluded_count} 个点)")

# 将预测结果分配到各个区间
bin_indices = []
for i in range(len(bins) - 1):
    mask = (valid_mean_preds >= bins[i]) & (valid_mean_preds < bins[i + 1])
    bin_indices.append(np.where(mask)[0])

# 计算各区间样本数量
bin_counts = [len(indices) for indices in bin_indices]
print("\n各砂厚区间原始分布:")
for i, count in enumerate(bin_counts):
    print(f"  {bin_labels[i]}: {count}个 ({count / len(valid_mean_preds) * 100:.1f}%)")

# 设定目标分布
target_distribution = np.array([0.4, 0.4, 0.20])

# 目标样本总数（最小200个）
target_total_samples = max(200, int(0.2 * len(valid_mean_preds)))
min_samples_per_bin = 15  # 每个区间的最小样本数

# 计算每个区间的目标样本数
target_bin_samples = np.round(target_distribution * target_total_samples).astype(int)
# 确保每个区间至少有最小样本数
target_bin_samples = np.maximum(target_bin_samples, min_samples_per_bin)
# 调整以匹配总目标
target_bin_samples = np.round(target_bin_samples / sum(target_bin_samples) * target_total_samples).astype(int)

print("\n目标样本分布:")
for i, count in enumerate(target_bin_samples):
    print(f"  {bin_labels[i]}: 目标{count}个样本 ({count / sum(target_bin_samples) * 100:.1f}%)")

# 初始阈值设置
base_thresholds = {
    "abs_threshold": 3.0,  # 绝对差异阈值
    "rel_threshold": 0.2,  # 相对差异阈值
    "std_threshold": 2.5,  # 标准差阈值
    "w_consistency": 0.6,  # 一致性权重
    "w_uncertainty": 0.4,  # 不确定性权重
    "min_quality": 0.6,  # 最低质量阈值
    "j_quality_weight": 0.6,  # J评分中质量的权重
    "j_dist_weight": 0.25,  # J评分中分布的权重
    "j_quantity_weight": 0.15,  # J评分中数量的权重
    "j_tolerance": 0.01,  # J评分容忍度
}

# 计算所有样本的初始质量分数
quality_scores = np.zeros(len(valid_mean_preds))

for i in range(len(valid_mean_preds)):
    quality_scores[i] = calculate_quality_score(
        valid_mean_preds[i],
        valid_max_diffs[i],
        valid_rel_diffs[i],
        valid_lasso_std[i],
        valid_ridge_std[i],
        base_thresholds,
    )

# 迭代优化样本选择
print("\n开始迭代优化样本选择...")

# 初始化选择的样本
selected_indices = np.array([], dtype=int)
max_iterations = 10
current_j = 0
current_thresholds = base_thresholds.copy()
min_iterations = 3  # 至少运行的迭代次数

# 记录每次迭代的状态，用于调试和分析
iteration_history = []

for iteration in range(max_iterations):
    print(f"\n迭代 {iteration + 1}/{max_iterations}")

    # 打印当前阈值
    print("当前阈值设置:")
    for key, value in current_thresholds.items():
        print(f"  {key}: {value:.4f}")

    # 1. 重新计算质量分数(使用当前阈值)
    for i in range(len(valid_mean_preds)):
        quality_scores[i] = calculate_quality_score(
            valid_mean_preds[i],
            valid_max_diffs[i],
            valid_rel_diffs[i],
            valid_lasso_std[i],
            valid_ridge_std[i],
            current_thresholds,
        )

    # 2. 计算当前各区间已选样本数
    current_bin_samples = [0] * len(bin_labels)
    if len(selected_indices) > 0:
        selected_values = valid_mean_preds[selected_indices]
        for i in range(len(bins) - 1):
            current_bin_samples[i] = np.sum((selected_values >= bins[i]) & (selected_values < bins[i + 1]))

    # 3. 计算各区间的覆盖率
    bin_ratios = np.zeros(len(bins) - 1)
    for i in range(len(bins) - 1):
        bin_ratios[i] = current_bin_samples[i] / target_bin_samples[i] if target_bin_samples[i] > 0 else 1.0

    # 按照样本充足率排序区间（优先处理最不足的区间）
    bin_priority = np.argsort(bin_ratios)

    # 记录本次迭代前的状态
    pre_iteration_state = {
        "selected_count": len(selected_indices),
        "j_score": current_j,
        "bin_ratios": bin_ratios.copy(),
        "thresholds": current_thresholds.copy(),
    }

    improved = False

    # 4. 逐个处理各区间
    for bin_idx in bin_priority:
        # 如果该区间已经足够，并且不是第一轮迭代，可以跳过
        if bin_ratios[bin_idx] >= 1.0 and iteration > 0:
            print(f"  区间 {bin_labels[bin_idx]} 已达到目标覆盖率 ({bin_ratios[bin_idx] * 100:.1f}%)，跳过")
            continue

        # 5. 动态调整质量阈值
        quality_threshold = get_dynamic_quality_threshold(
            bin_idx=bin_idx, sample_ratio=bin_ratios[bin_idx], base_threshold=current_thresholds["min_quality"]
        )

        # 打印原始阈值和动态调整后的阈值，增强可理解性
        print(
            f"  处理区间 {bin_labels[bin_idx]} - 原始质量阈值={current_thresholds['min_quality']:.2f}, "
            f"动态调整后={quality_threshold:.2f}, 覆盖率={bin_ratios[bin_idx] * 100:.1f}%"
        )

        # 6. 获取该区间的候选样本
        candidates = bin_indices[bin_idx]

        # 排除已选择的样本
        candidates = np.setdiff1d(candidates, selected_indices)

        if len(candidates) == 0:
            print(f"  区间 {bin_labels[bin_idx]} 没有更多可用样本")
            continue

        # 7. 按质量排序
        candidates = candidates[np.argsort(-quality_scores[candidates])]

        # 8. 筛选符合质量要求的样本
        qualified_candidates = candidates[quality_scores[candidates] >= quality_threshold]

        if len(qualified_candidates) == 0:
            print(f"  区间 {bin_labels[bin_idx]} 没有符合质量要求(>{quality_threshold:.2f})的样本，跳过该区间")
            continue  # 不再尝试降低质量标准

        # 9. 计算需要添加的样本数
        needed = target_bin_samples[bin_idx] - current_bin_samples[bin_idx]
        to_add = min(needed, len(qualified_candidates))

        if to_add <= 0:
            print(f"  区间 {bin_labels[bin_idx]} 不需要添加更多样本")
            continue

        print(f"  尝试向区间 {bin_labels[bin_idx]} 添加 {to_add} 个样本 (质量阈值>{quality_threshold:.2f})")

        # 10. 添加样本并评估J值变化
        new_selected = (
            np.concatenate([selected_indices, qualified_candidates[:to_add]])
            if len(selected_indices) > 0
            else qualified_candidates[:to_add]
        )

        new_j = calculate_j_score(
            new_selected, quality_scores, valid_mean_preds, target_distribution, bins, current_thresholds
        )

        # 11. 评估J值变化
        j_tolerance = current_thresholds["j_tolerance"]
        j_change = new_j - current_j

        # 如果J值提高或在容忍范围内，接受新样本
        if new_j > current_j - j_tolerance or len(selected_indices) == 0:
            selected_indices = new_selected
            current_j = new_j
            improved = True

            print(f"  【接受】添加了 {to_add} 个样本，新J值: {current_j:.4f}")
            if j_change < 0:
                print(f"  注意: 接受了J值轻微下降({j_change:.4f})以获取更多样本")
        else:
            print(f"  【拒绝】J值下降超出容忍范围，不添加样本")

    # 记录本次迭代后的状态
    post_iteration_state = {"selected_count": len(selected_indices), "j_score": current_j, "improved": improved}

    iteration_history.append({**pre_iteration_state, **post_iteration_state})

    # 调整阈值，为下一轮迭代做准备
    current_thresholds = adjust_thresholds(current_thresholds, iteration + 1, bin_ratios)

    # 如果达到最小迭代次数且本轮迭代没有改进，考虑提前结束
    if iteration >= min_iterations and not improved:
        print(f"  已完成{iteration + 1}轮迭代且本轮无改进，提前结束")
        break

    # 打印当前状态
    if len(selected_indices) > 0:
        selected_values = valid_mean_preds[selected_indices]
        current_hist, _ = np.histogram(selected_values, bins=bins)
        print(f"  当前已选择 {len(selected_indices)} 个样本，J值 = {current_j:.4f}")
        for i in range(len(bins) - 1):
            coverage = current_hist[i] / target_bin_samples[i] * 100 if target_bin_samples[i] > 0 else 0
            print(
                f"    {bin_labels[i]}: {current_hist[i]}个 ({current_hist[i] / len(selected_indices) * 100:.1f}%), 目标覆盖率: {coverage:.1f}%"
            )

# 将选中的索引映射回原始索引
final_selected_indices_orig = valid_unlabeled_indices[selected_indices]

# 创建最终掩码
final_mask = np.zeros(len(unlabeled_indices), dtype=bool)
final_mask[np.where(np.isin(unlabeled_indices, final_selected_indices_orig))[0]] = True

# 统计最终结果
print("\n=== 最终样本选择结果 ===")
final_hist, _ = np.histogram(mean_preds[final_mask], bins=bins)
print(f"总计选择样本: {np.sum(final_mask)}个")
for i in range(len(bins) - 1):
    original_count = bin_counts[i]
    final_count = final_hist[i]
    coverage = final_count / target_bin_samples[i] * 100 if target_bin_samples[i] > 0 else 0
    print(
        f"砂厚范围 {bin_labels[i]}: {final_count}个 ({final_count / np.sum(final_mask) * 100:.1f}%) | "
        f"原始: {original_count}个 | 目标覆盖率: {coverage:.1f}%"
    )

# 统计最终质量分数
final_quality = quality_scores[selected_indices]
print(f"\n样本质量统计:")
print(f"  平均质量分数: {np.mean(final_quality):.4f}")
print(f"  最低质量分数: {np.min(final_quality):.4f}")
print(f"  最高质量分数: {np.max(final_quality):.4f}")

# 打印迭代历史摘要
print("\n迭代历史摘要:")
for i, hist in enumerate(iteration_history):
    print(f"  迭代 {i + 1}: 样本数 {hist['selected_count']}, J值 {hist['j_score']:.4f}")

# 创建伪标记数据
X_pseudo = X_unlabeled[final_mask]
y_pseudo = mean_preds[final_mask]

# 获取优化样本在原始seismic_samples中的索引
optimized_orig_indices = unlabeled_indices[final_mask]

# 添加预测结果到seismic_samples
seismic_samples["Fused_Pred"] = np.nan
seismic_samples["Lasso_Pred"] = np.nan
seismic_samples["Ridge_Pred"] = np.nan
seismic_samples["Mean_Pred"] = np.nan
seismic_samples["Max_Diff"] = np.nan
seismic_samples["Quality_Score"] = np.nan
seismic_samples["Is_Selected"] = False
seismic_samples["Excluded_By_Well"] = False  # 新增列：被真实井点排除

# 填充预测结果
seismic_samples.loc[unlabeled_indices, "Fused_Pred"] = fused_pred_unlabeled
seismic_samples.loc[unlabeled_indices, "Lasso_Pred"] = lasso_pred_unlabeled
seismic_samples.loc[unlabeled_indices, "Ridge_Pred"] = ridge_pred_unlabeled
seismic_samples.loc[unlabeled_indices, "Mean_Pred"] = mean_preds
seismic_samples.loc[unlabeled_indices, "Max_Diff"] = max_diffs
seismic_samples.loc[unlabeled_indices, "Quality_Score"] = np.nan  # 先设为NaN
seismic_samples.loc[valid_unlabeled_indices, "Quality_Score"] = quality_scores  # 只填充有效点的质量分数

# 标记被真实井点排除的点
excluded_indices = unlabeled_indices[~np.isin(unlabeled_indices, valid_unlabeled_indices)]
seismic_samples.loc[excluded_indices, "Excluded_By_Well"] = True

# 标记最终选择的点
seismic_samples.loc[optimized_orig_indices, "Is_Selected"] = True

# 统计并报告负值数量
neg_count = (seismic_samples["Mean_Pred"] < 0).sum()
if neg_count > 0:
    print(f"\n注意: 有 {neg_count} 个负的砂厚预测值已被替换为0")

# 将负数的Mean_Pred值置为0
seismic_samples["Mean_Pred"] = seismic_samples["Mean_Pred"].clip(lower=0)

# 可视化优化前后的分布变化
plt.figure(figsize=(12, 6))

# 原始分布
bin_percents = np.array(bin_counts) / sum(bin_counts) * 100
plt.bar(np.arange(len(bin_labels)) - 0.2, bin_percents, width=0.4, label="原始分布", color="lightblue")

# 优化后分布
optimized_percents = final_hist / sum(final_hist) * 100
plt.bar(np.arange(len(bin_labels)) + 0.2, optimized_percents, width=0.4, label="优化后分布", color="orange")

# 添加目标分布线
target_percents = target_distribution / sum(target_distribution) * 100
plt.plot(np.arange(len(bin_labels)), target_percents, "r--", label="目标分布", linewidth=2)

plt.xlabel("砂厚区间")
plt.ylabel("百分比 (%)")
plt.title("样本分布优化前后对比")
plt.xticks(np.arange(len(bin_labels)), bin_labels)
plt.legend()
plt.grid(True, linestyle="--", alpha=0.7)
plt.savefig(os.path.join(output_dir, "distribution_optimization.png"), dpi=300, bbox_inches="tight")
plt.show()

# 保存预测结果
seismic_samples.to_csv(os.path.join(output_dir, "seismic_samples_with_predictions.csv"), index=False)
print(f"\n预测结果已保存至 {os.path.join(output_dir, 'seismic_samples_with_predictions.csv')}")

# 保存优化后的虚拟井数据
optimized_samples = seismic_samples[seismic_samples["Is_Selected"] == True].copy()
optimized_samples.to_csv(os.path.join(output_dir, "optimized_pseudo_wells.csv"), index=False)
print(f"优化后的虚拟井数据已保存至 {os.path.join(output_dir, 'optimized_pseudo_wells.csv')}")

# 计算质量-分布的J指标
final_j = calculate_j_score(
    selected_indices, quality_scores, valid_mean_preds, target_distribution, bins, current_thresholds
)
print(f"\n最终优化J评分: {final_j:.4f}")

## 虚拟井展示


In [ ]:
# 准备小工区地震数据（使用已筛选的data_H6_2_attr_filtered）
# 1. 首先创建融合属性
print("为整个小工区地震数据创建融合属性...")

# 标准化数据
scaler = StandardScaler()
attr_data = seismic_attr_filtered[selected_features].copy()
attr_data_scaled = scaler.fit_transform(attr_data)

# 使用之前定义的create_fused_attribute函数创建融合属性
fused_attr = create_fused_attribute(attr_data_scaled, selected_features, correlation_weights)

# 将融合属性添加到地震数据中
seismic_attr_filtered["Fused_Attribute"] = fused_attr

# 2. 准备真实井点数据
real_wells = well_attr_filtered.copy()

# 3. 准备虚拟井点数据（使用之前优化选择的虚拟井）
pseudo_wells = seismic_samples[seismic_samples["Is_Selected"] == True].copy()

# 可视化融合属性分布与井点位置
visualize_attribute_map(
    data_points=seismic_attr_filtered,
    attribute_name="Fused_Attribute",
    attribute_label="地震融合属性值",
    real_wells=real_wells,
    pseudo_wells=pseudo_wells,
    target_column="Sand Thickness",
    output_dir=output_dir,
    filename_prefix="fused_attribute",
    class_thresholds=[0.1, 10],
    figsize=(16, 14),
    dpi=300,
    cmap="viridis",
    point_size=140,
    well_size=200,
)

print("融合属性分布与井点位置可视化完成")

## SVR、随机森林（限制树深）、XGBoost（控制学习率和复杂度）


In [ ]:
# 设置随机种子以确保结果可复现
np.random.seed(42)

print("======== 三模型联合预测开始 ========")

# 步骤1: 准备数据 - 加载所有井点数据和虚拟井点数据
# 使用所有井点数据(未筛选)
all_wells = well_attr.copy()  # 使用所有井点，而不是筛选后的井点
print(f"全部井点数据数量: {len(all_wells)}")

# 加载虚拟井点数据
pseudo_wells = pd.read_csv(os.path.join(output_dir, "optimized_pseudo_wells.csv"))
print(f"加载了 {len(pseudo_wells)} 个虚拟井点数据")

# 确定共同的特征列
common_features = selected_features.copy()
print(f"使用的特征: {common_features}")

# 确保井点数据有所需的特征列
all_wells_valid = all_wells.dropna(subset=common_features + ["Sand Thickness"])
print(f"有效井点数据数量: {len(all_wells_valid)}")

# 从真实井点中提取特征和目标
X_real = all_wells_valid[common_features].values
y_real = all_wells_valid["Sand Thickness"].values

# 从虚拟井点中提取特征和目标
X_pseudo = pseudo_wells[common_features].values
y_pseudo = pseudo_wells["Mean_Pred"].values  # 使用平均预测作为目标值

# 提取虚拟井的质量分数作为权重
pseudo_quality_weights = pseudo_wells["Quality_Score"].values

# 确保权重在合理范围内
if np.any(np.isnan(pseudo_quality_weights)):
    print("警告：虚拟井质量分数中存在NaN值，将使用均值填充")
    pseudo_quality_weights = np.nan_to_num(pseudo_quality_weights, nan=np.nanmean(pseudo_quality_weights))

# 确保权重为正数且进行适当缩放
pseudo_quality_weights = np.clip(pseudo_quality_weights, 0.2, 1.0)  # 限制最小权重为0.2

print(f"虚拟井质量分数统计：")
print(f"  - 平均值: {np.mean(pseudo_quality_weights):.4f}")
print(f"  - 最小值: {np.min(pseudo_quality_weights):.4f}")
print(f"  - 最大值: {np.max(pseudo_quality_weights):.4f}")

# 标准化真实井点数据(用于留一法)
scaler_real = StandardScaler()
X_real_scaled = scaler_real.fit_transform(X_real)

# 创建留一法交叉验证对象
loocv = LeaveOneOut()

# 步骤2: 设置模型参数网格搜索(保留原有参数)
# SVR参数网格
param_grid_svr = {"C": [0.1, 1, 10], "gamma": ["scale", 0.01, 0.1], "epsilon": [0.1, 0.2], "kernel": ["rbf"]}

# 随机森林参数网格
param_grid_rf = {
    "n_estimators": [50, 100],
    "max_depth": [3, 4, 5],
    "min_samples_leaf": [3, 5],
    "max_features": ["sqrt", "log2"],
}

# XGBoost参数网格
param_grid_xgb = {
    "n_estimators": [50, 100],
    "max_depth": [3, 4, 5],
    "learning_rate": [0.01, 0.05, 0.1],
    "subsample": [0.8],
    "colsample_bytree": [0.8],
    "min_child_weight": [3, 5],
}

# 使用网格搜索找到最佳参数
print("\n执行SVR参数网格搜索...")
svr = SVR()
grid_search_svr = GridSearchCV(svr, param_grid_svr, cv=5, scoring="neg_mean_squared_error", n_jobs=-1)
# 使用所有真实井点数据进行参数调优
grid_search_svr.fit(X_real_scaled, y_real)
best_svr_params = grid_search_svr.best_params_
print(f"SVR最佳参数: {best_svr_params}")

print("\n执行随机森林参数网格搜索...")
rf = RandomForestRegressor(random_state=42)
grid_search_rf = GridSearchCV(rf, param_grid_rf, cv=5, scoring="neg_mean_squared_error", n_jobs=-1)
grid_search_rf.fit(X_real_scaled, y_real)
best_rf_params = grid_search_rf.best_params_
print(f"随机森林最佳参数: {best_rf_params}")

print("\n执行XGBoost参数网格搜索...")
xgb = XGBRegressor(random_state=42)
grid_search_xgb = GridSearchCV(xgb, param_grid_xgb, cv=5, scoring="neg_mean_squared_error", n_jobs=-1)
grid_search_xgb.fit(X_real_scaled, y_real)
best_xgb_params = grid_search_xgb.best_params_
print(f"XGBoost最佳参数: {best_xgb_params}")


# 步骤3: 使用留一法交叉验证评估模型性能
# 创建SVR预测函数
def loocv_predict_with_pseudo(X_real, y_real, X_pseudo, y_pseudo, pseudo_weights, model_creator, model_params):
    """使用留一法交叉验证评估模型，同时利用虚拟井点进行训练"""
    loocv = LeaveOneOut()
    predictions = np.zeros_like(y_real)

    for train_idx, test_idx in loocv.split(X_real):
        # 分割数据
        X_real_train, X_real_test = X_real[train_idx], X_real[test_idx]
        y_real_train, y_real_test = y_real[train_idx], y_real[test_idx]

        # 合并真实训练样本和虚拟样本
        X_train = np.vstack((X_real_train, X_pseudo))
        y_train = np.concatenate((y_real_train, y_pseudo))

        # 标准化数据
        scaler = StandardScaler()
        X_train_scaled = scaler.fit_transform(X_train)
        X_test_scaled = scaler.transform(X_real_test)

        # 构建权重
        train_weights = np.concatenate(
            [
                np.ones(len(X_real_train)) * 2.0,  # 真实样本权重为2
                pseudo_weights,  # 虚拟样本使用质量分数作为权重
            ]
        )

        # 创建并训练模型
        if model_creator == "svr":
            # SVR特征数量限制
            svr_features = min(3, X_train_scaled.shape[1])
            X_train_svr = X_train_scaled[:, :svr_features]
            X_test_svr = X_test_scaled[:, :svr_features]

            model = SVR(**model_params)
            model.fit(X_train_svr, y_train, sample_weight=train_weights)
            predictions[test_idx] = model.predict(X_test_svr)
        else:
            model = model_creator(**model_params)
            model.fit(X_train_scaled, y_train, sample_weight=train_weights)
            predictions[test_idx] = model.predict(X_test_scaled)

    return predictions


print("\n执行留一法交叉验证...")

# SVR模型预测
print("留一法评估SVR模型...")
# 注意SVR处理特征数量的限制
svr_features = min(3, len(common_features))
X_real_svr = X_real[:, :svr_features]
X_pseudo_svr = X_pseudo[:, :svr_features]

# SVR预测
svr_pred = loocv_predict_with_pseudo(
    X_real_svr, y_real, X_pseudo_svr, y_pseudo, pseudo_quality_weights, "svr", best_svr_params
)

# 随机森林预测
print("留一法评估随机森林模型...")
rf_params = best_rf_params.copy()
rf_params["random_state"] = 42
rf_pred = loocv_predict_with_pseudo(
    X_real, y_real, X_pseudo, y_pseudo, pseudo_quality_weights, RandomForestRegressor, rf_params
)

# XGBoost预测
print("留一法评估XGBoost模型...")
xgb_params = best_xgb_params.copy()
xgb_params["random_state"] = 42
xgb_pred = loocv_predict_with_pseudo(
    X_real, y_real, X_pseudo, y_pseudo, pseudo_quality_weights, XGBRegressor, xgb_params
)


# 步骤4: 评估留一法交叉验证结果
def evaluate_loocv_results(y_true, y_pred, model_name):
    """评估留一法交叉验证结果"""
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    r2 = r2_score(y_true, y_pred)
    mae = mean_absolute_error(y_true, y_pred)

    print(f"\n{model_name} 留一法交叉验证评估:")
    print(f"  - RMSE: {rmse:.4f}")
    print(f"  - R²: {r2:.4f}")
    print(f"  - MAE: {mae:.4f}")

    # 绘制真实值vs预测值散点图
    plt.figure(figsize=(10, 6))
    plt.scatter(y_true, y_pred, alpha=0.7)
    plt.plot([min(y_true), max(y_true)], [min(y_true), max(y_true)], "r--")
    plt.xlabel("真实值")
    plt.ylabel("预测值")
    plt.title(f"{model_name}: 留一法交叉验证 真实值 vs 预测值")
    plt.grid(True, alpha=0.3)
    plt.savefig(
        os.path.join(output_dir, f"{model_name.lower().replace(' ', '_')}_loocv_pred_vs_true.png"),
        dpi=300,
        bbox_inches="tight",
    )
    plt.show()

    return {"rmse": rmse, "r2": r2, "mae": mae}


# 评估各模型留一法结果
svr_results = evaluate_loocv_results(y_real, svr_pred, "SVR")
rf_results = evaluate_loocv_results(y_real, rf_pred, "随机森林")
xgb_results = evaluate_loocv_results(y_real, xgb_pred, "XGBoost")

# 比较模型性能
loocv_models_comparison = pd.DataFrame(
    {
        "模型": ["SVR", "随机森林", "XGBoost"],
        "RMSE": [svr_results["rmse"], rf_results["rmse"], xgb_results["rmse"]],
        "R²": [svr_results["r2"], rf_results["r2"], xgb_results["r2"]],
        "MAE": [svr_results["mae"], rf_results["mae"], xgb_results["mae"]],
    }
)

print("\n留一法交叉验证模型性能比较:")
print(loocv_models_comparison)

# 保存比较结果
loocv_models_comparison.to_csv(os.path.join(output_dir, "loocv_model_comparison.csv"), index=False)

# 步骤5: 训练最终模型
print("\n训练最终模型...")

# 选择性能最佳的模型类型
best_model_idx = loocv_models_comparison["R²"].idxmax()
best_model_name = loocv_models_comparison.loc[best_model_idx, "模型"]
print(f"最佳模型: {best_model_name}")

# 合并所有数据用于训练最终模型
X_combined = np.vstack((X_real, X_pseudo))
y_combined = np.concatenate((y_real, y_pseudo))

# 创建权重数组 - 真实井点权重为2.0，虚拟井点使用质量分数
combined_weights = np.concatenate(
    [
        np.ones(len(X_real)) * 2.0,  # 真实井点权重为2
        pseudo_quality_weights,  # 虚拟井点权重为质量分数
    ]
)

# 标准化特征
scaler_final = StandardScaler()
X_combined_scaled = scaler_final.fit_transform(X_combined)

# 根据最佳模型类型训练最终模型
if best_model_name == "SVR":
    # SVR特征限制
    svr_features = min(3, len(common_features))
    X_combined_svr = X_combined_scaled[:, :svr_features]

    final_model = SVR(**best_svr_params)
    final_model.fit(X_combined_svr, y_combined, sample_weight=combined_weights)
elif best_model_name == "随机森林":
    rf_params = best_rf_params.copy()
    rf_params["random_state"] = 42
    final_model = RandomForestRegressor(**rf_params)
    final_model.fit(X_combined_scaled, y_combined, sample_weight=combined_weights)
else:  # XGBoost
    xgb_params = best_xgb_params.copy()
    xgb_params["random_state"] = 42
    final_model = XGBRegressor(**xgb_params)
    final_model.fit(X_combined_scaled, y_combined, sample_weight=combined_weights)

# 步骤6: 为整个工区生成预测
print("\n使用最终模型为整个工区生成预测...")
seismic_data = seismic_attr_filtered.copy()
X_seismic = seismic_data[common_features].fillna(seismic_data[common_features].mean())
X_seismic_scaled = scaler_final.transform(X_seismic)

# 根据模型类型生成预测
if best_model_name == "SVR":
    X_seismic_svr = X_seismic_scaled[:, :svr_features]
    predictions = final_model.predict(X_seismic_svr)
else:
    predictions = final_model.predict(X_seismic_scaled)

# 将预测结果添加到地震数据
seismic_data["LOOCV_Predicted_Sand_Thickness"] = predictions

# 将负值预测设为0
if (predictions < 0).any():
    neg_count = (predictions < 0).sum()
    print(f"注意: {neg_count} 个负的砂厚预测值已被替换为0")
    seismic_data["LOOCV_Predicted_Sand_Thickness"] = seismic_data["LOOCV_Predicted_Sand_Thickness"].clip(lower=0)

# 保存预测结果
seismic_data.to_csv(os.path.join(output_dir, "seismic_with_loocv_predictions.csv"), index=False)

# 可视化最终预测结果
visualize_attribute_map(
    data_points=seismic_data,
    attribute_name="LOOCV_Predicted_Sand_Thickness",
    attribute_label=f"砂厚预测值(米) - {best_model_name}模型",
    real_wells=well_attr_filtered,
    pseudo_wells=None,
    target_column="Sand Thickness",
    output_dir=output_dir,
    filename_prefix=f"loocv_final_prediction_{best_model_name.lower()}",
    class_thresholds=[0.1, 10],
    figsize=(14, 14),
    dpi=300,
    cmap="viridis",
    point_size=150,
    well_size=200,
)

# 如果是树模型，可视化特征重要性
if best_model_name in ["随机森林", "XGBoost"]:
    print("\n分析特征重要性...")

    # 获取特征重要性
    importances = final_model.feature_importances_

    # 创建特征重要性DataFrame
    feature_importance_df = pd.DataFrame({"特征": common_features, "重要性": importances})
    feature_importance_df = feature_importance_df.sort_values("重要性", ascending=False)

    # 绘制特征重要性
    plt.figure(figsize=(10, 6))
    plt.barh(feature_importance_df["特征"], feature_importance_df["重要性"])
    plt.xlabel("特征重要性")
    plt.title(f"{best_model_name}模型特征重要性")
    plt.tight_layout()
    plt.savefig(
        os.path.join(output_dir, f"loocv_{best_model_name.lower()}_feature_importance.png"),
        dpi=300,
        bbox_inches="tight",
    )
    plt.show()

    # 保存特征重要性
    feature_importance_df.to_csv(
        os.path.join(output_dir, f"loocv_{best_model_name.lower()}_feature_importance.csv"), index=False
    )

print("\n留一法交叉验证与质量分数加权建模完成。所有结果已保存到输出目录。")